In [33]:
import pandas as pd
import numpy as np
import os
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

pio.templates.default = "plotly_white"
pd.options.display.float_format = '{:.4f}'.format

Cargando Data

In [ ]:
ruta = r"C:\Users\HP\Desktop\Practica\Proyectos\DataExtraction_Riesgos\Data_Extraction\Raw_Files\sura_fondo.xls"
try:
    df = pd.read_html(ruta, header=0)[0]
except:
    df = pd.read_html(ruta, header=0, encoding='utf-8')[0]
# Limpiando espacios en nombres de columnas
df.columns = df.columns.astype(str).str.strip()

# TRANSFORMACIÓN DE DATA A FORMATO
cols_fechas = ["Fecha Información", "Fecha Inicio"]
for col in cols_fechas:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], dayfirst=True, errors='coerce')

cols_numericas = ["Valor Cuota", "Patrimonio", "Participes", "N° Cuotas", "Tipo Cambio"]
for col in cols_numericas:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', ''), errors='coerce')
        
#  ORDENANDO Y LIMPIANDO
df = df.dropna(subset=["Fecha Información"])
df = df.sort_values("Fecha Información").reset_index(drop=True)

In [49]:
df_risk = df[['Fecha Información', 'Valor Cuota']].copy()
df_risk.columns = ['Fecha', 'Precio']

df_risk = df_risk[df_risk['Precio'] > 0].copy()
df_risk = df_risk.set_index('Fecha').sort_index()

# CÁLCULO DE RENTABILIDAD
# Log Retorno: ln(Pt / Pt-1)
df_risk['Retorno'] = np.log(df_risk['Precio'] / df_risk['Precio'].shift(1))
returns = df_risk['Retorno'].dropna()

# CÁLCULO DE MÉTRICAS
vol_diaria = returns.std()
vol_anual = vol_diaria * np.sqrt(252)

# VaR Histórico
var_99 = np.percentile(returns, 1)    
var_975 = np.percentile(returns, 2.5) 

# REPORTE FINANCIERO
print("╔══════════════════════════════════════════════════════════════╗")
print("║   REPORTE DE RIESGO: SURA ULTRA CASH (2022-2024)             ║")
print("╚══════════════════════════════════════════════════════════════╝")
print(f" Registros válidos: {len(returns)}")
print(f" Rentabilidad Total Acumulada: {(df_risk['Precio'].iloc[-1]/df_risk['Precio'].iloc[0] - 1):.2%}")
print(f" Volatilidad Anualizada: {vol_anual:.4%} (Extremadamente baja)")
print("-" * 60)
print("MÉTRICAS DE VaR (Riesgo de Pérdida Diaria):")
print(f" VaR 99.0%:  {var_99:.6%}")
print(f" VaR 97.5%:  {var_975:.6%}")
print("-" * 60)

╔══════════════════════════════════════════════════════════════╗
║   REPORTE DE RIESGO: SURA ULTRA CASH (2022-2024)             ║
╚══════════════════════════════════════════════════════════════╝
 Registros válidos: 1094
 Rentabilidad Total Acumulada: 18.84%
 Volatilidad Anualizada: 0.0775% (Extremadamente baja)
------------------------------------------------------------
MÉTRICAS DE VaR (Riesgo de Pérdida Diaria):
 VaR 99.0%:  0.000437%
 VaR 97.5%:  0.005180%
------------------------------------------------------------


In [50]:
# Gráfico de Retornos (Histograma)
fig = go.Figure()
fig.add_trace(go.Histogram(
    x=returns, nbinsx=100, 
    name='Retornos Diarios', marker_color='#2ca02c'
))
# Línea de VaR
fig.add_vline(x=var_99, line_width=2, line_color="red", line_dash="dash")
fig.add_annotation(x=var_99, y=10, text="VaR 99%", showarrow=True, arrowhead=1, ax=0, ay=-40)

fig.update_layout(
    title='<b>Distribución de Retornos Diarios</b> | Sura Ultra Cash',
    xaxis_title='Rentabilidad Diaria',
    yaxis_title='Días'
)
fig.show()

In [51]:
# Curva de Crecimiento 
fig_nav = px.line(df_risk, x=df_risk.index, y='Precio', 
                  title='<b>Crecimiento del Capital </b> | Tendencia Creciente')
fig_nav.update_traces(line_color='#2ca02c', line_width=3) 
fig_nav.update_layout(yaxis_title="Valor Cuota (S/.)", hovermode="x unified")
fig_nav.show()

In [52]:
# --- CÁLCULO DE DRAWDOWN
# 1. Calcular el valor acumulado (Wealth Index)
wealth_index = (1 + returns).cumprod()
# 2. Calcular el pico histórico acumulado hasta el momento
previous_peaks = wealth_index.cummax()
# 3. Calcular la caída porcentual desde ese pico
drawdown = (wealth_index - previous_peaks) / previous_peaks

# --- GRÁFICO: UNDERWATER PLOT (DRAWDOWN) ---
# ¿Qué tan abajo estamos desde el máximo histórico?
fig_dd = go.Figure()

fig_dd.add_trace(go.Scatter(
    x=drawdown.index, 
    y=drawdown,
    fill='tozeroy',
    mode='lines',
    line=dict(color='red', width=1),
    name='Drawdown'
))

fig_dd.update_layout(
    title='<b>Maximum Drawdown',
    yaxis_title='Caída %',
    xaxis_title='Fecha',
    yaxis=dict(tickformat=".2%"), 
    hovermode="x unified"
)

fig_dd.show()

max_dd = drawdown.min()
print(f" MÁXIMO DRAWDOWN HISTÓRICO: {max_dd:.6%}")
if max_dd == 0:
    print("   (El fondo nunca ha caído respecto a su máximo anterior. Solidez total).")

 MÁXIMO DRAWDOWN HISTÓRICO: -0.039836%


Conclusiones

El Fondo Sura Ultra Cash Soles demuestra, ser un instrumento idóneo para la Gestión de Tesorería Corporativa y para perfiles de inversión Conservadores. Su comportamiento descarta la exposición a riesgos de mercado significativos.

Esta conclusión se sustenta en los siguientes hallazgos:

- Perfil de Riesgo Nulo (VaR Positivo): El análisis de Valor en Riesgo (VaR) confirma la seguridad del activo en múltiples escenarios. Al 99% de confianza, el VaR es de +0.000437%, y al 97.5% de confianza, mejora a +0.005180%. Ambos resultados certifican que el fondo no proyecta pérdidas nominales.

- Estabilidad (Volatilidad): La volatilidad anual de 0.0775% es inmaterial comparada con cualquier otro activo de renta fija o variable. Esto valida que el fondo inmuniza el capital contra la volatilidad de mercado.

- Resiliencia Operativa (Drawdown): La prueba de estrés histórico (Maximum Drawdown) reveló una caída máxima de -0.04% en tres años, con una recuperación inmediata. Desde 2023 a la fecha, el Drawdown ha sido 0.00%, lo que demuestra una solidez absoluta ante choques de liquidez recientes.

- Eficiencia en Captura de Tasas: Con una rentabilidad acumulada del 18.84% (aprox. 6.28% anual promedio), el instrumento ha logrado trasladar eficientemente las tasas de referencia altas del BCRP al inversor, superando la inflación promedio del periodo y preservando el poder adquisitivo de la caja.